In [38]:
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from collections import Counter
import shutil
import uuid

# Load the trained model
model = load_model("yoga_video_classifierV2.h5")

# Must match training class order
class_names = ['Balassana', 'Bhungassan', 'Padmasan', 'Parvatassan', 'Savasan', 'LeftTrikonasanas', 'RightTrikonasanas']


# 🧱 1. Use existing extraction function
def extract_frames_single_video(video_path, output_dir='temp_frames', frame_rate=30):
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_rate == 0:
            frame_name = f"{uuid.uuid4().hex}.jpg"
            frame_path = os.path.join(output_dir, frame_name)
            cv2.imwrite(frame_path, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    return saved_count


# 🧠 2. Predict from frames
def predict_from_frames(frame_dir, img_size=(224, 224)):
    predictions = []

    for fname in os.listdir(frame_dir):
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        path = os.path.join(frame_dir, fname)
        image = cv2.imread(path)
        image = cv2.resize(image, img_size)
        image = img_to_array(image) / 255.0
        image = np.expand_dims(image, axis=0)

        pred = model.predict(image, verbose=0)
        predicted_class = np.argmax(pred)
        predictions.append(predicted_class)

    if not predictions:
        return "No frames processed."

    most_common_class = Counter(predictions).most_common(1)[0][0]
    return class_names[most_common_class]


# 🧪 3. Full prediction pipeline
def predict_video_class(video_path, frame_rate=30):
    temp_dir = "temp_frames"

    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)

    frame_count = extract_frames_single_video(video_path, output_dir=temp_dir, frame_rate=frame_rate)
    print(f"Extracted {frame_count} frames from video.")

    result = predict_from_frames(temp_dir)

    shutil.rmtree(temp_dir)  # Clean up
    return result


In [32]:
video_path = r"Dataset\Padmasan\Padmasan2.mp4"

In [33]:
# video_file = "test_video.mp4"
predicted_class = predict_video_class(video_path, frame_rate=30)
print("Predicted Asana:", predicted_class)


Extracted 12 frames from video.
Predicted Asana: Parvatassan


In [40]:
video_path = r"Dataset\Padmasan\Padmasan3.mp4"
# video_file = "test_video.mp4"
predicted_class = predict_video_class(video_path, frame_rate=30)
print("Predicted Asana:", predicted_class)


Extracted 11 frames from video.
Predicted Asana: Parvatassan


In [41]:
video_path = r"Dataset\Balassana\Balassana18.mp4"
# video_file = "test_video.mp4"
predicted_class = predict_video_class(video_path, frame_rate=30)
print("Predicted Asana:", predicted_class)


Extracted 9 frames from video.
Predicted Asana: Balassana
